In [2]:
import os
import pandas as pd
from pathlib import Path
import shutil

/tmp/ipykernel_444158/2791841992.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
df = pd.read_excel("/home/sakshmeno/Documents/GitHub/C-RNN-approach/Labels/Labelling_Prateek_Guillermo.xlsx")

In [4]:
file_start = {}
for i in range(len(df['File'])):
    if pd.isnull(df['File'][i]) == False:
        file_start[df['File'][i]] = i

BOTTOM_LINE = file_start['TOTAL NO. OF FILES']
file_list = list(file_start.keys())[:-1]
file_vulnerabilities = {}

for i in range(len(file_list)-1):
    vulnerable_lines=[]
    for j in range(file_start[file_list[i]],file_start[file_list[i+1]]-1):
        if (df['Comment'][j].split('|')[1].split()[0]) == "BRANCH:":
            if (pd.isna(df.at[j,'False Positive'])):
                vulnerable_lines.append(df['Comment'][j].split('|')[0].split()[1])
        if (not(pd.isna(df.at[j,'Lines Missed']))):
            if (not(pd.isna(df.at[j,'Branch.1']))):
                line = df['Lines Missed'][j].split()
                if (not(line[0].isnumeric())):
                    vulnerable_lines.append(line[1].strip(":"))
                else:
                    vulnerable_lines.append(line[0].strip(":"))
    vulnerable_lines.sort()

    file_vulnerabilities[file_list[i]]=vulnerable_lines

file_list = file_list[:-1]

In [6]:
os.chdir("/home/sakshmeno/Documents/GitHub/C-RNN-approach/dataset")
dataset = {}
for file in file_list:
    with open(file) as dataset_file_obj:
        dataset[file] = dataset_file_obj.readlines()

In [17]:
dataset['AX3_Firmware main.c'][222]#[int(file_vulnerabilities['AX3_Firmware main.c'][0])]

'if(FSInit()!=TRUE)                          // Initialize the filesystem for reading\t\t\t\t\t\t\n'

In [14]:
[int(file_vulnerabilities['AX3_Firmware main.c'][0])]

[223]

In [105]:
insecure_lines = {}
secure_lines = {}

for file in file_list:
    codeLines = []
    file_insecure_lines = []
    file_secure_lines = []
    with open(file) as c_file_obj:
        codeLines = (c_file_obj.readlines())
    for line_number in range(1,len(codeLines)):
        if str(line_number) in file_vulnerabilities[file]:
            file_insecure_lines.append(codeLines[int(line_number)-1].strip("\t"))
        else:
            file_secure_lines.append(codeLines[int(line_number)-1].strip("\t"))

    insecure_lines[file] = file_insecure_lines
    secure_lines[file] = file_secure_lines

In [106]:
def create_unique_filename(filename, destination):
    
    os.chdir(destination)
    if not os.path.exists(filename):
        return filename

    base_name, extension = os.path.splitext(filename)
    counter = 1

    while True:
        alternate_filename = f"{base_name}_{counter}{extension}"

        if not os.path.exists(alternate_filename):
            return alternate_filename

        counter += 1

In [107]:

secure_file_name = 'SECURE_DATA.txt'
unique_secure_file_name = create_unique_filename(secure_file_name, "/home/sakshmeno/Documents/GitHub/C-RNN-approach/Labels")

insecure_file_name = 'INSECURE_DATA.txt'
unique_insecure_file_name = create_unique_filename(insecure_file_name, "/home/sakshmeno/Documents/GitHub/C-RNN-approach/Labels")

dataset_file_name = 'DATASET.txt'
unique_dataset_file_name = create_unique_filename(dataset_file_name, "/home/sakshmeno/Documents/GitHub/C-RNN-approach/Labels")

for file in file_list:
    
    with open(unique_secure_file_name,'a') as secure_c_code:
        secure_c_code.writelines(secure_lines[file])
    
    with open(unique_insecure_file_name,'a') as insecure_c_code:
        insecure_c_code.writelines(insecure_lines[file])

    with open(unique_dataset_file_name,'a') as dataset_c_code:
        dataset_c_code.writelines(dataset[file])